In [1]:
import numpy as np
import pandas
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import power_transform

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

### Define data directory and file name to load

In [2]:
day_of_week = 'Tuesday'

data_directory='../data/raw'
file_name=day_of_week+'-WorkingHours.pcap_ISCX.csv'

data_file=os.path.join(data_directory,file_name)

Tried the package loader from github clean code

In [3]:
# from anomalyzer.config import data_file

### Load the data frame

In [6]:
df=pandas.read_csv(data_file,skipinitialspace=True,low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445909 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445909 non-null int64
Flow Duration                  445909 non-null int64
Total Fwd Packets              445909 non-null int64
Total Backward Packets         445909 non-null int64
Total Length of Fwd Packets    445909 non-null int64
Total Length of Bwd Packets    445909 non-null int64
Fwd Packet Length Max          445909 non-null int64
Fwd Packet Length Min          445909 non-null int64
Fwd Packet Length Mean         445909 non-null float64
Fwd Packet Length Std          445909 non-null float64
Bwd Packet Length Max          445909 non-null int64
Bwd Packet Length Min          445909 non-null int64
Bwd Packet Length Mean         445909 non-null float64
Bwd Packet Length Std          445909 non-null float64
Flow Bytes/s                   445708 non-null object
Flow Packets/s                 445909 non-null object
Flow IAT Mean                  

Some of the features have the character '/' in them, which might be problematic. Therefore, such features are renamed to be on the safe side.

In [25]:
df.rename(columns={'Flow Bytes/s':'Flow Bytes per sec',
                   'Flow Packets/s':'Flow Packets per sec',
                   'Fwd Packets/s':'Fwd Packets per sec',
                   'Bwd Packets/s':'Bwd Packets per sec',
                   'Fwd Avg Bytes/Bulk':'Fwd Avg Bytes per Bulk',
                   'Fwd Avg Packets/Bulk':'Fwd Avg Packets per Bulk',
                   'Bwd Avg Bytes/Bulk':'Bwd Avg Bytes per Bulk',
                   'Bwd Avg Packets/Bulk':'Bwd Avg Packets per Bulk',
                   'Down/Up Ratio':'Down Up Ratio'
                  },inplace=True)

We also quickly check the distibution of attacks in the data set.

In [26]:
df['Label'].value_counts(normalize=True)

BENIGN         0.968973
FTP-Patator    0.017802
SSH-Patator    0.013225
Name: Label, dtype: float64

It looks like 96.90% of the row in our dataset are BENIGN communications and the remaining 3.10% are attacks on the network.

## Data Cleaning

### Remove NaN values from the data frame

In [27]:
clean_df=df.dropna()
clean_df.dtypes.value_counts()

int64      54
float64    22
object      3
dtype: int64

In [28]:
clean_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,88,640,7,4,440,358,220,0,62.857143,107.349008,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,88,900,9,4,600,2944,300,0,66.666667,132.287566,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1205,7,4,2776,2830,1388,0,396.571429,677.274651,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,88,511,7,4,452,370,226,0,64.571429,110.276708,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,773,9,4,612,2944,306,0,68.000000,134.933317,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [29]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445708 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445708 non-null int64
Flow Duration                  445708 non-null int64
Total Fwd Packets              445708 non-null int64
Total Backward Packets         445708 non-null int64
Total Length of Fwd Packets    445708 non-null int64
Total Length of Bwd Packets    445708 non-null int64
Fwd Packet Length Max          445708 non-null int64
Fwd Packet Length Min          445708 non-null int64
Fwd Packet Length Mean         445708 non-null float64
Fwd Packet Length Std          445708 non-null float64
Bwd Packet Length Max          445708 non-null int64
Bwd Packet Length Min          445708 non-null int64
Bwd Packet Length Mean         445708 non-null float64
Bwd Packet Length Std          445708 non-null float64
Flow Bytes per sec             445708 non-null object
Flow Packets per sec           445708 non-null object
Flow IAT Mean                  

There are still two 'object' type features in the dataset: Flow Bytes per sec and Flow Packets per sec

### We now need to remove 'Infinity' values typed as string from the data.

In [30]:
index_to_drop=clean_df.index[(clean_df['Flow Packets per sec']=='Infinity')|(clean_df['Flow Bytes per sec']=='Infinity')]
print(index_to_drop)

Int64Index([   167,    172,   2439,   4058,  15297,  22891,  28757,  33942,
             35342,  38478,  41582,  44806,  45262,  48751,  61676,  63183,
             67315,  88127,  93289,  93595,  93975, 111843, 118883, 120246,
            121444, 121536, 129524, 136265, 137752, 137844, 138921, 147048,
            150093, 153792, 154300, 154805, 157396, 159859, 169208, 196971,
            198370, 200593, 206184, 210835, 212289, 215264, 215271, 216393,
            216570, 216749, 216776, 216805, 218521, 222605, 225099, 227162,
            247620, 253996, 257193, 258160, 268813, 276059, 276562],
           dtype='int64')


In [31]:
final_df=clean_df.drop(index_to_drop)

In [32]:
final_df['Flow Bytes per sec']=final_df['Flow Bytes per sec'].astype('float')
final_df['Flow Packets per sec']=final_df['Flow Packets per sec'].astype('float')

In [33]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445645 entries, 0 to 445908
Data columns (total 79 columns):
Destination Port               445645 non-null int64
Flow Duration                  445645 non-null int64
Total Fwd Packets              445645 non-null int64
Total Backward Packets         445645 non-null int64
Total Length of Fwd Packets    445645 non-null int64
Total Length of Bwd Packets    445645 non-null int64
Fwd Packet Length Max          445645 non-null int64
Fwd Packet Length Min          445645 non-null int64
Fwd Packet Length Mean         445645 non-null float64
Fwd Packet Length Std          445645 non-null float64
Bwd Packet Length Max          445645 non-null int64
Bwd Packet Length Min          445645 non-null int64
Bwd Packet Length Mean         445645 non-null float64
Bwd Packet Length Std          445645 non-null float64
Flow Bytes per sec             445645 non-null float64
Flow Packets per sec           445645 non-null float64
Flow IAT Mean                

Finally, we notice there are two columns called **Fwd Header Length** and **Fwd Header Length.1**. We check if they are identical and drop one of them if true.

In [34]:
if sum(final_df['Fwd Header Length']-final_df['Fwd Header Length.1'])==0:
    final_df.drop(['Fwd Header Length.1'],axis=1,inplace=True)

Now that our data frame is clean, we can pickle it into the cleaned data directory.

In [15]:
final_df.to_pickle('../data/cleaned/'+day_of_week+'_clean.pkl')

## Data Processing

Let's start by plotting histograms of the features we have in our dataset. All files are saved into the ./figures directory for later referencing.

In [35]:
features=final_df.columns.values

In [17]:
for index, feature in enumerate(features,1):
    fig = plt.figure()
    final_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/original_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

A cursory look at the histograms show us two things:

1. Some of the values seem to have a single spike indicating we need to look for features with a single value. Since these features carry no information, we should identify and remove them from the data frame.

2. The range of values for most of the attributes is quite large, resulting in largely skewed distributions. A log transform should be applied.

### 1.  Find the columns that have a single value and drop them from the dataframe

In [36]:
columns_to_drop=[]
for feature in features:
    if final_df[feature].unique().size==1:
        columns_to_drop.append(feature)
columns_to_drop

['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Fwd Avg Bytes per Bulk',
 'Fwd Avg Packets per Bulk',
 'Fwd Avg Bulk Rate',
 'Bwd Avg Bytes per Bulk',
 'Bwd Avg Packets per Bulk',
 'Bwd Avg Bulk Rate']

In [37]:
reduced_df=final_df.drop(columns_to_drop,axis=1)

In [38]:
reduced_features=reduced_df.columns.values
reduced_features

array(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd PSH Flags', 'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count',
       'RST Flag Count', 'PS

10 features have been removed, leaving us with 78-10=68 features. 

Let's save their histograms in a separate directory. This will help us choose features for further transformations to come from the remaining feature set.

In [21]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    reduced_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/reduced_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

In [39]:
reduced_df.to_pickle('../data/processed/Reduced/'+day_of_week+'_processed.pkl')

### 2. Skewness transforms

The histograms also showed us that the range of values for some of the features was very large, resulting in largely skewed values. Now let's try to do a transform on the remaining features to try to reshape their distribution to a more "normal" looking form.

In [31]:
transform_candidates=np.array(['Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes per sec',
       'Flow Packets per sec', 'Flow IAT Mean', 'Flow IAT Std',
       'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean',
       'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total',
       'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min',
       'Fwd Header Length', 'Bwd Header Length',
       'Fwd Packets per sec', 'Bwd Packets per sec', 'Min Packet Length',
       'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
       'Packet Length Variance',  'Down Up Ratio',
       'Average Packet Size', 'Avg Fwd Segment Size',
       'Avg Bwd Segment Size', 'Subflow Fwd Packets', 'Subflow Fwd Bytes',
       'Subflow Bwd Packets', 'Subflow Bwd Bytes',
       'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
       'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean',
       'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std',
       'Idle Max', 'Idle Min'])

There are 10 (one being the class label) features excluded from the transformation list. These features are binary valued, therefore transformation does not make sense.

In [32]:
no_transform_features=np.setdiff1d(reduced_features,transform_candidates)
no_transform_features

array(['ACK Flag Count', 'Destination Port', 'ECE Flag Count',
       'FIN Flag Count', 'Fwd PSH Flags', 'Label', 'PSH Flag Count',
       'RST Flag Count', 'SYN Flag Count', 'URG Flag Count'], dtype=object)

Since there are both positive and negative values across attributes as well as zero values, a log transform does not work. The standard transformer in this case is the 'Yeo-Johnson' transformation. 

Read more at: https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-transformer

In [33]:
transformed_df=pandas.DataFrame()

for feature in reduced_features:
    if feature in transform_candidates:
        transformed_df.loc[:,feature]=power_transform(reduced_df.loc[:,feature].values.reshape(-1,1),method='yeo-johnson')
    else:
        transformed_df.loc[:,feature]=reduced_df.loc[:,feature]

In [34]:
transformed_df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,691406.000000,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,...,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05,6.914060e+05
mean,5683.120410,4.794734e-16,1.983007e-16,-3.120853e-16,-8.813367e-17,-1.450259e-16,8.543704e-16,-2.052067e-16,6.465328e-16,1.052342e-17,...,-2.499313e-17,3.275251e-15,2.078376e-16,1.479856e-17,-4.580977e-16,2.104685e-16,1.802136e-16,9.832823e-17,3.656889e-16,-2.045490e-16
std,15722.340146,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,0.000000,-1.909622e+00,-1.525682e+00,-1.526514e+00,-1.628975e+00,-1.328361e+00,-1.578769e+00,-8.259742e-01,-1.686634e+00,-8.929728e-01,...,-1.312139e+00,-5.235059e+00,-6.682325e-01,-2.623876e-01,-6.674971e-01,-6.689406e-01,-7.049336e-01,-2.817000e-01,-7.049363e-01,-7.049090e-01
25%,53.000000,-9.846914e-01,-5.547218e-01,-5.373832e-01,-7.581861e-01,-1.328361e+00,-9.066696e-01,-8.259742e-01,-8.259480e-01,-8.929728e-01,...,-1.312139e+00,-1.068829e+00,-6.682325e-01,-2.623876e-01,-6.674971e-01,-6.689406e-01,-7.049336e-01,-2.817000e-01,-7.049363e-01,-7.049090e-01
50%,80.000000,-9.318346e-02,-5.547218e-01,-3.518820e-02,-7.480104e-04,9.975822e-03,-1.023874e-01,-8.259742e-01,2.757057e-01,-8.929728e-01,...,5.796008e-02,8.335120e-01,-6.682325e-01,-2.623876e-01,-6.674971e-01,-6.689406e-01,-7.049336e-01,-2.817000e-01,-7.049363e-01,-7.049090e-01
75%,443.000000,1.207579e+00,9.604115e-01,8.599716e-01,6.967329e-01,1.205817e+00,9.249767e-01,1.286335e+00,5.117634e-01,1.138914e+00,...,6.349606e-01,8.335120e-01,1.405647e+00,-2.623876e-01,1.393732e+00,1.424703e+00,1.381598e+00,-2.817000e-01,1.385160e+00,1.365750e+00
max,65487.000000,1.279739e+00,2.647183e+00,4.305271e+00,6.520178e+00,5.114445e+00,4.023305e+00,1.825639e+00,5.537890e+00,1.851665e+00,...,2.767409e+00,4.868940e+00,1.736163e+00,3.812521e+00,1.743204e+00,1.731721e+00,1.450165e+00,3.550408e+00,1.449153e+00,1.452538e+00


We save the histograms of the transformed features once again to confirm that transformations have worked as intended.

In [35]:
for index, feature in enumerate(reduced_features,1):
    fig = plt.figure()
    transformed_df[feature].hist()
    plt.title(feature+' Distribution')
    plt.xlabel('Value')
    plt.ylabel('Count')
    plt.savefig('./figures/'+day_of_week+'/transformed_features/'+str(index)+'_'+feature+'_Histogram.png')
    plt.close(fig)

We finally pickle the resulting data frame after processing into the processed data directory.

In [36]:
transformed_df.to_pickle('../data/processed/Normalized/'+day_of_week+'_processed.pkl')